In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow import keras
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q "/content/drive/MyDrive/train_data.zip"

In [4]:
img_gen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    validation_split=0.2
)

In [10]:
BATCH_SIZE = 40
train_ds = img_gen.flow_from_directory(directory = '/content/train_data' , batch_size = BATCH_SIZE ,
                                       shuffle = True , class_mode = 'sparse' ,
                                       target_size = (299,299) , subset = 'training' , seed = 42)
val_ds = img_gen.flow_from_directory(directory = '/content/train_data' , batch_size = BATCH_SIZE ,
                                     shuffle = False  , class_mode = 'sparse' ,
                                     target_size = (299,299) , subset = 'validation', seed = 42)

Found 7994 images belonging to 10 classes.
Found 1996 images belonging to 10 classes.


In [11]:
from keras.src.saving.saving_lib import optimizer
from keras.applications.inception_resnet_v2 import InceptionResNetV2

num_classes = 10
base_model_1 = InceptionResNetV2(weights='imagenet' , include_top = False)
x = base_model_1.output
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation='relu')(x)
predictions = keras.layers.Dense(num_classes, activation='softmax')(x)
model_1 = keras.Model(inputs=base_model_1.input, outputs=predictions)
model_1.compile(loss = 'sparse_categorical_crossentropy' ,
                     optimizer = 'sgd' , metrics = ['accuracy'])
for layer in model_1.layers[:775] :
  layer.trainable = False
for layer in model_1.layers[775:] :
  layer.trainable = True
check_inc_res = keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/inc_res_v2_1.h5', monitor='val_accuracy', save_best_only=False)

In [12]:
history = model_1.fit(train_ds , validation_data = val_ds , epochs = 25 , callbacks=[check_inc_res] )

Epoch 1/25
200/200 [==============================] - ETA: 0s - loss: 1.8706 - accuracy: 0.3852

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


200/200 [==============================] - 393s 2s/step - loss: 1.8706 - accuracy: 0.3852 - val_loss: 1.4889 - val_accuracy: 0.5020
Epoch 2/25
200/200 [==============================] - 319s 2s/step - loss: 1.3281 - accuracy: 0.5630 - val_loss: 1.5066 - val_accuracy: 0.4970
Epoch 3/25
200/200 [==============================] - 321s 2s/step - loss: 1.0826 - accuracy: 0.6420 - val_loss: 1.1926 - val_accuracy: 0.6057
Epoch 4/25
200/200 [==============================] - 318s 2s/step - loss: 0.8691 - accuracy: 0.7138 - val_loss: 1.1725 - val_accuracy: 0.6137
Epoch 5/25
200/200 [==============================] - 336s 2s/step - loss: 0.7034 - accuracy: 0.7713 - val_loss: 1.1803 - val_accuracy: 0.6197
Epoch 6/25
200/200 [==============================] - 342s 2s/step - loss: 0.5279 - accuracy: 0.8335 - val_loss: 1.3329 - val_accuracy: 0.6373
Epoch 7/25
200/200 [==============================] - 320s 2s/step - loss: 0.3790 - accuracy: 0.8844 - val_loss: 1.2027 - val_accuracy: 0.6463
Epoch 8/25